In [9]:
# Ý tưởng:
# Biểu diễn mỗi ký tự bằng one-hot
# Ở mỗi bước t:
#     Thực hiện tính công thức trạng thái ẩn
#     Đầu ra đưa qua softmax
#     loss là cross-entropy loss
# Backpropagation: Tính gradient từ T -> 1, cộng dồn qua thời gian
# Có gradient clipping tránh nổ gradient
import numpy as np

In [10]:
def one_hot(idx, vocab_size):
    # vector v có vocab_size hàng và 1 cột
    v = np.zeros((vocab_size, 1))
    # Cả hàng idx sẽ toàn giá trị 1.0
    v[idx] = 1.0
    return v
def softmax(z):
    exp = np.exp(z)
    return exp / np.sum(exp)


In [11]:
v = np.zeros((3, 2))
v[1] = 1.0
v

array([[0., 0.],
       [1., 1.],
       [0., 0.]])

In [ ]:
# RNN thuần
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Khởi tạo trọng số gồm value + và -
        s = 0.01
        # ma trận gồm hidden_size hàng, input_size cột
        self.Wxh = np.random.randn(hidden_size, input_size) * s
        # ma trận gồm hidden_size hàng, hidden_size cột
        self.Whh = np.random.randn(hidden_size, hidden_size) * s
        self.bh = np.zeros((hidden_size, 1))
        # ma trận gồm output_size hàng, hidden_size cột
        self.Why = np.random.randn(output_size, hidden_size) * s
        self.by = np.zeros((output_size, 1))

        #Adam
        self.m = {k: np.zeros_like(v) for k, v in self.params().items()}
        self.v = {k: np.zeros_like(v) for k, v in self.params().items()}
        self.t = 0

    def params(self):
        return {
            "Wxh": self.Wxh, "Whh": self.Whh, "bh": self.bh,
            "Why": self.Why, "by": self.by
        }
    def forward(self, inputs, prev_hidden_state):
        # inputs: list chỉ số của ký tự [x1, x2,...]
        # prev_hidden_state : hidden state ban đầu
        # return cache(xs, hs, os, ps) và hidden last
        # xs, hs, os, ps chứa các xt, ht, ot, pt trong công thức của RNN
        xs, hs, os, ps = {}, {}, {}, {}
        # hidden state trước t = 0
        hs[-1] = np.copy(prev_hidden_state)
        loss = 0.0
        for t, ix in enumerate(inputs):
            xs[t] = one_hot(ix, self.input_size)
            hs[t] = np.tanh(self.Wxh @ xs[t] + self.Whh @ hs[t-1] + self.bh)
            os[t] = self.Why @ hs[t] + self.by
            ps[t] = softmax(os[t])
        cache = (xs, hs, os, ps)
        # trả về hidden state cuối cùng làm input cho batch tiếp theo
        # trả về cache để tính backpropagation
        return cache, hs[len(inputs)-1]
    def loss_and_grads(self, inputs, targets, prev_hidden_state):
        (xs, hs, os, ps), hlast = self.forward(inputs, prev_hidden_state=prev_hidden_state)
        loss = 0.0
        # Tính loss
        for t in range(len(inputs)):
            loss += -np.log(ps[t][targets[t],0] + 1e-12)
        # Khởi tạo grads, tạo ra ma trận 0 giống với các ma trận trong ngoặc
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dbh = np.zeros_like(self.bh)
        dWhy = np.zeros_like(self.Why)
        dby = np.zeros_like(self.by)
        dh_next = np.zeros(hs[0])

        # Backpropagation through time
        for t in reversed(range(len(inputs))):
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1.0
            dWhy += dy @ hs[t].T
            dby += dy

            dh = self.Why.T @ dy + dh_next
            dh_raw = (1 - hs[t] * hs[t]) * dh      # tanh'
            dbh  += dh_raw
            dWxh += dh_raw @ xs[t].T
            dWhh += dh_raw @ hs[t-1].T
            dh_next = self.Whh.T @ dh_raw
        
        # clipping tránh nổ gradient
        for dparam in [dWxh, dWhh, dWhy, dby]:
            np.clip(dparam, -5, 5, out=dparam)
        
        grads = {"Wxh":dWxh, "Whh":dWhh, "bh":dbh, "Why": dWhy, "by": dby}
        # loss tổng lỗi của chuỗi
        # grads dictionary chứa các gradient để cập nhật trọng số
        # hlast là hidden cuối cùng làm prev hidden state cho batch tiếp theo
        return loss, grads, hlast
    
    # Cập nhật trọng số dùng Adam
    def step(self, grads, lr=1e-2, beta1=0.9, beta2=0.999, eps=1e-8):
        # Số lần update (để hiệu chỉnh bias correction)
        self.t += 1
        # Cập nhật trọng số
        for k in self.params().keys():
            g = grads[k]
            self.m[k] = beta1 * self.m[k] + (1-beta1) * g
            self.v[k] = beta2 * self.v[k] + (1-beta2) * (g * g)

            m_hat = self.m[k] / (1 - beta1**self.t)
            v_hat = self.v[k] / (1 - beta2**self.t)

            self.params()[k] -= lr * m_hat / (np.sqrt(v_hat) + eps)
        
    

In [13]:
wxh = np.random.randn(3, 5)
wxh

array([[-0.58193921, -2.34964456, -0.07479449,  1.34274883,  0.21664595],
       [-0.13370841, -1.02788999,  1.43742827, -0.54657575,  0.69828735],
       [ 0.65384239,  0.05486468,  0.2140975 , -1.62696669,  1.65284889]])

In [14]:
Wxh = np.random.randn(3, 5)
print(Wxh)
dwxh = np.zeros_like(Wxh)
print("---------")
print(dwxh)

[[-0.92710378  1.92306457  0.7018871   1.05275921  0.4326244 ]
 [-0.36930456 -0.08725843  1.72317063 -0.36386441  0.08908316]
 [ 0.03981613 -2.25674317  0.52674794 -0.98753193  1.21862034]]
---------
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [15]:
s = 0.01
input_size = 4
hidden_size = 3
output_size = input_size
# ma trận gồm hidden_size hàng, input_size cột
Wxh = np.random.randn(hidden_size, input_size) * s
print("Wxh (hidden size, input size): \n",Wxh,"\n")
# ma trận gồm hidden_size hàng, hidden_size cột
Whh = np.random.randn(hidden_size, hidden_size) * s
print("Whh (hidden size, hidden size):\n",Whh, "\n")
bh = np.zeros((hidden_size, 1))
print("bh (hidden size):\n",bh, "\n")
# ma trận gồm output_size hàng, hidden_size cột
Why = np.random.randn(output_size, hidden_size) * s
print("Why (output size, hidden size): \n",Why, "\n")
by = np.zeros((output_size, 1))
print("by (output size): \n",by, "\n")

Wxh (hidden size, input size): 
 [[-0.0246867  -0.01388412 -0.01374919 -0.00191876]
 [-0.01397797  0.00897254 -0.01009596 -0.00666188]
 [ 0.00431107  0.00232595 -0.01185167 -0.00438465]] 

Whh (hidden size, hidden size):
 [[-0.00915568  0.00113654  0.01755194]
 [-0.00070391  0.00096014 -0.00965476]
 [ 0.00554101 -0.00473686 -0.00104836]] 

bh (hidden size):
 [[0.]
 [0.]
 [0.]] 

Why (output size, hidden size): 
 [[-0.0064983   0.01020995 -0.00232772]
 [-0.0039177   0.0006422  -0.00051127]
 [-0.00115289 -0.0075742  -0.001058  ]
 [-0.00283878 -0.01293675 -0.0020665 ]] 

by (output size): 
 [[0.]
 [0.]
 [0.]
 [0.]] 



In [16]:
inputs = [1, 2, 3]
input_size = 4
xs, hs, os, ps = {}, {}, {}, {}
hs[-1] = np.zeros((hidden_size,1))
for t, ix in enumerate(inputs):
    xs[t] = one_hot(ix, input_size)
    hs[t] = np.tanh(Wxh @ xs[t] + Whh @ hs[t-1] + bh)
    os[t] = Why @ hs[t] + by
    ps[t] = softmax(os[t])
    print(f"Giá trị thứ {t} của input là:\n{xs[t]}\nCó hidden state thứ {t} là:\n{hs[t]}")
    print(f"Output thứ {t} của input thứ {t} là:\n {os[t]}\nCó xác suất:\n {ps[t]}")
    print("---------------------------------------------------------")

Giá trị thứ 0 của input là:
[[0.]
 [1.]
 [0.]
 [0.]]
Có hidden state thứ 0 là:
[[-0.01388323]
 [ 0.0089723 ]
 [ 0.00232595]]
Output thứ 0 của input thứ 0 là:
 [[ 1.76410078e-04]
 [ 5.89631536e-05]
 [-5.44130023e-05]
 [-8.14676239e-05]]
Có xác suất:
 [[0.25003789]
 [0.25000852]
 [0.24998018]
 [0.24997341]]
---------------------------------------------------------
Giá trị thứ 1 của input là:
[[0.]
 [0.]
 [1.]
 [0.]]
Có hidden state thứ 1 là:
[[-0.01357023]
 [-0.01009969]
 [-0.01197296]]
Output thứ 1 của input thứ 1 là:
 [[1.29358802e-05]
 [5.27994594e-05]
 [1.04809396e-04]
 [1.93922163e-04]]
Có xác suất:
 [[0.24998045]
 [0.24999042]
 [0.25000342]
 [0.2500257 ]]
---------------------------------------------------------
Giá trị thứ 2 của input là:
[[0.]
 [0.]
 [0.]
 [1.]]
Có hidden state thứ 2 là:
[[-0.00201614]
 [-0.00654633]
 [-0.00439942]]
Output thứ 2 của input thứ 2 là:
 [[-4.34956033e-05]
 [ 5.94386033e-06]
 [ 5.65621678e-05]
 [ 9.95030386e-05]]
Có xác suất:
 [[0.24998172]
 [0.249994